# SP Simulation of WSCC 9-bus System with 4th Order Synchronous Generator and PSAT Validation

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np

import re
import math
import os
import subprocess
import requests
import urllib.request


def download_grid_data(name, url):
    with open(name, "wb") as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)


# %matplotlib widget

PEAK1PH_TO_RMS3PH = np.sqrt(3.0 / 2.0)

name_exec = "SP_WSCC9bus_SGReducedOrderVBR"

order_names_list = ["4th"]  # ['3rd', '4th', '6th']
order_options_list = ["sgType=4"]  # ['sgType=3', 'sgType=4', 'sgType=6b']
sim_names_list = [name_exec + "_" + order_name for order_name in order_names_list]
num_orders = len(order_names_list)

timestep = 1e-3
duration = 30

view_time_interval = [0.1, 1.0]

root_path = (
    subprocess.Popen(["git", "rev-parse", "--show-toplevel"], stdout=subprocess.PIPE)
    .communicate()[0]
    .rstrip()
    .decode("utf-8")
)

path_exec = root_path + "/build/dpsim/examples/cxx/"

cim_file = "WSCC-09_Dyn_Fourth"

cim_url = "https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_Dyn_Fourth/WSCC-09_Dyn_Fourth"
download_grid_data(cim_file + "_EQ.xml", cim_url + "_EQ.xml")
download_grid_data(cim_file + "_TP.xml", cim_url + "_TP.xml")
download_grid_data(cim_file + "_SV.xml", cim_url + "_SV.xml")
download_grid_data(cim_file + "_DI.xml", cim_url + "_DI.xml")

psat_results_url = "https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/WSCC-9bus/d_009_fault_dpsim_4th_order.out"
psat_results_file = "d_009_fault_dpsim_4th_order.out"
urllib.request.urlretrieve(psat_results_url, psat_results_file)

## Run Simulation

In [ ]:
for order_idx in range(num_orders):
    sim = subprocess.Popen(
        [
            path_exec + name_exec,
            "--name",
            sim_names_list[order_idx],
            "--timestep",
            str(timestep),
            "--duration",
            str(duration),
            cim_file + "_DI.xml",
            cim_file + "_EQ.xml",
            cim_file + "_SV.xml",
            cim_file + "_TP.xml",
            "--option",
            order_options_list[order_idx],
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
    )
    print(sim.communicate()[0].decode())

## Simulation

In [ ]:
ts_dpsim = []
for order_idx in range(num_orders):
    path = "logs/" + sim_names_list[order_idx] + "/"
    logName = sim_names_list[order_idx]
    logFilename = path + logName + ".csv"
    print(logFilename)
    ts_dpsim.append(rt.read_timeseries_dpsim(logFilename))

## Validation with PSAT

In [ ]:
syngen_power_name_dpsim_list = ["GEN1.Te", "GEN2.Te", "GEN3.Te"]
syngen_power_name_psat_list = ["p_Syn_1", "p_Syn_2", "p_Syn_3"]

syngen_omega_name_dpsim_list = ["GEN1.omega", "GEN2.omega", "GEN3.omega"]
syngen_omega_name_psat_list = ["omega_Syn_1", "omega_Syn_2", "omega_Syn_3"]

syngen_delta_name_dpsim_list = ["GEN1.delta", "GEN2.delta", "GEN3.delta"]
syngen_delta_name_psat_list = ["delta_Syn_1", "delta_Syn_2", "delta_Syn_3"]

bus_volt_name_dpsim_list = [
    "BUS1.V",
    "BUS2.V",
    "BUS3.V",
    "BUS4.V",
    "BUS5.V",
    "BUS6.V",
    "BUS7.V",
    "BUS8.V",
    "BUS9.V",
]
bus_volt_name_psat_list = [
    "V_Bus 1",
    "V_Bus 2",
    "V_Bus 3",
    "V_Bus 4",
    "V_Bus 5",
    "V_Bus 6",
    "V_Bus 7",
    "V_Bus 8",
    "V_Bus 9",
]
bus_angle_name_psat_list = [
    "theta_Bus 1",
    "theta_Bus 2",
    "theta_Bus 3",
    "theta_Bus 4",
    "theta_Bus 5",
    "theta_Bus 6",
    "theta_Bus 7",
    "theta_Bus 8",
    "theta_Bus 9",
]

timeseries_names_psat = (
    syngen_power_name_psat_list
    + syngen_omega_name_psat_list
    + syngen_delta_name_psat_list
    + bus_volt_name_psat_list
    + bus_angle_name_psat_list
)

ts_psat = []
for order_idx in range(num_orders):
    ts_psat.append(rt.read_timeseries_PSAT(psat_results_file, timeseries_names_psat))

## Rotor speeds

In [ ]:
for order_idx in range(num_orders):
    plt.figure(figsize=(12, 9))
    # plt.subplot(num_orders,1,order_idx+1)
    for syngen_omega_name_dpsim in syngen_omega_name_dpsim_list:
        plt.plot(
            ts_dpsim[order_idx][syngen_omega_name_dpsim].time,
            ts_dpsim[order_idx][syngen_omega_name_dpsim].values,
            label=syngen_omega_name_dpsim
            + ", "
            + order_names_list[order_idx]
            + " (dpsim)",
        )
    for syngen_omega_name_psat in syngen_omega_name_psat_list:
        plt.plot(
            ts_psat[order_idx][syngen_omega_name_psat].time,
            ts_psat[order_idx][syngen_omega_name_psat].values,
            label=syngen_omega_name_psat
            + ", "
            + order_names_list[order_idx]
            + " (psat)",
            linestyle="--",
        )

    # plt.ylim([0.99,1.02])
    plt.xlim(view_time_interval)

    plt.xlabel("time (s)")
    plt.ylabel("mechanical speed (p.u)")
    plt.legend()

## Rotor angle

In [ ]:
plt.figure(figsize=(12, 9))
for order_idx in range(num_orders):
    plt.subplot(num_orders, 1, order_idx + 1)
    for syngen_delta_name_dpsim in syngen_delta_name_dpsim_list:
        plt.plot(
            ts_dpsim[order_idx][syngen_delta_name_dpsim].time,
            ts_dpsim[order_idx][syngen_delta_name_dpsim].values / np.pi * 180,
            label=syngen_delta_name_dpsim
            + ", "
            + order_names_list[order_idx]
            + " (dpsim)",
        )
    for syngen_delta_name_psat in syngen_delta_name_psat_list:
        plt.plot(
            ts_psat[order_idx][syngen_delta_name_psat].time,
            ts_psat[order_idx][syngen_delta_name_psat].values / np.pi * 180,
            label=syngen_delta_name_psat
            + ", "
            + order_names_list[order_idx]
            + " (psat)",
            linestyle="--",
        )

    plt.xlim(view_time_interval)
    plt.xlabel("time (s)")
    plt.ylabel("rotor angle (rad)")
    plt.legend()

## Rotor angles with reference angle of GEN1

In [ ]:
for order_idx in range(num_orders):
    plt.figure(figsize=(12, 9))
    # plt.subplot(num_orders,1,order_idx+1)
    for syngen_delta_name_dpsim in syngen_delta_name_dpsim_list:
        plt.plot(
            ts_dpsim[order_idx][syngen_delta_name_dpsim].time,
            (
                ts_dpsim[order_idx][syngen_delta_name_dpsim].values
                - ts_dpsim[order_idx]["GEN1.delta"].values
                + ts_dpsim[order_idx]["GEN1.delta"].values[0]
            )
            / np.pi
            * 180,
            label=syngen_delta_name_dpsim
            + ", "
            + order_names_list[order_idx]
            + " (dpsim)",
        )
    for syngen_delta_name_psat in syngen_delta_name_psat_list:
        plt.plot(
            ts_psat[order_idx][syngen_delta_name_psat].time,
            (
                ts_psat[order_idx][syngen_delta_name_psat].values
                - ts_psat[order_idx]["delta_Syn_3"].values
                + ts_psat[order_idx]["delta_Syn_3"].values[0]
            )
            / np.pi
            * 180,
            label=syngen_delta_name_psat
            + ", "
            + order_names_list[order_idx]
            + " (psat)",
            linestyle="--",
        )

    plt.xlim(view_time_interval)
    plt.xlabel("time (s)")
    plt.ylabel("rotor angle (rad)")
    plt.legend()

## Bus voltages

In [ ]:
plt.figure(figsize=(12, 12))
for order_idx in range(num_orders):
    plt.subplot(num_orders, 1, order_idx + 1)
    for bus_volt_name_dpsim in bus_volt_name_dpsim_list:
        if bus_volt_name_dpsim == "BUS1.V":
            plt.plot(
                ts_dpsim[order_idx][bus_volt_name_dpsim].time,
                ts_dpsim[order_idx][bus_volt_name_dpsim].abs().values / 16.5e3,
                label=bus_volt_name_dpsim
                + ", "
                + order_names_list[order_idx]
                + " (dpsim)",
                color="C" + str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)),
            )
        elif bus_volt_name_dpsim == "BUS2.V":
            plt.plot(
                ts_dpsim[order_idx][bus_volt_name_dpsim].time,
                ts_dpsim[order_idx][bus_volt_name_dpsim].abs().values / 18e3,
                label=bus_volt_name_dpsim
                + ", "
                + order_names_list[order_idx]
                + " (dpsim)",
                color="C" + str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)),
            )
        elif bus_volt_name_dpsim == "BUS3.V":
            plt.plot(
                ts_dpsim[order_idx][bus_volt_name_dpsim].time,
                ts_dpsim[order_idx][bus_volt_name_dpsim].abs().values / 13.8e3,
                label=bus_volt_name_dpsim
                + ", "
                + order_names_list[order_idx]
                + " (dpsim)",
                color="C" + str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)),
            )
        else:
            plt.plot(
                ts_dpsim[order_idx][bus_volt_name_dpsim].time,
                ts_dpsim[order_idx][bus_volt_name_dpsim].abs().values / 230e3,
                label=bus_volt_name_dpsim
                + ", "
                + order_names_list[order_idx]
                + " (dpsim)",
                color="C" + str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)),
            )
    for bus_volt_name_psat in bus_volt_name_psat_list:
        plt.plot(
            ts_psat[order_idx][bus_volt_name_psat].time,
            ts_psat[order_idx][bus_volt_name_psat].values,
            label=bus_volt_name_psat + ", " + order_names_list[order_idx] + " (psat)",
            linestyle="--",
            color="C" + str(bus_volt_name_psat_list.index(bus_volt_name_psat)),
        )

    plt.xlim(view_time_interval)
    plt.xlabel("time (s)")
    plt.ylabel("voltage (p.u.)")
    plt.legend()

## Bus angles

In [ ]:
plt.figure(figsize=(12, 12))
for order_idx in range(num_orders):
    plt.subplot(num_orders, 1, order_idx + 1)
    for bus_volt_name_dpsim in bus_volt_name_dpsim_list:
        plt.plot(
            ts_dpsim[order_idx][bus_volt_name_dpsim].time,
            ts_dpsim[order_idx][bus_volt_name_dpsim].phase().values / 180 * np.pi
            - ts_dpsim[order_idx]["BUS1.V"].phase().values / 180 * np.pi,
            label=bus_volt_name_dpsim + ", " + order_names_list[order_idx] + " (dpsim)",
            color="C" + str(bus_volt_name_dpsim_list.index(bus_volt_name_dpsim)),
        )
    for bus_angle_name_psat in bus_angle_name_psat_list:
        plt.plot(
            ts_psat[order_idx][bus_angle_name_psat].time,
            ts_psat[order_idx][bus_angle_name_psat].values,
            label=bus_angle_name_psat + ", " + order_names_list[order_idx] + " (psat)",
            linestyle="--",
            color="C" + str(bus_angle_name_psat_list.index(bus_angle_name_psat)),
        )

    plt.xlim(view_time_interval)
    plt.xlabel("time (s)")
    plt.ylabel("angle (rad)")
    plt.legend()

## SG active power

In [ ]:
for order_idx in range(num_orders):
    plt.figure(figsize=(12, 9))
    # plt.subplot(num_orders,1,order_idx+1)
    for syngen_power_name_dpsim in syngen_power_name_dpsim_list:
        plt.plot(
            ts_dpsim[order_idx][syngen_power_name_dpsim].time,
            ts_dpsim[order_idx][syngen_power_name_dpsim].values,
            label=syngen_power_name_dpsim
            + ", "
            + order_names_list[order_idx]
            + " (dpsim)",
        )
    for syngen_power_name_psat in syngen_power_name_psat_list:
        plt.plot(
            ts_psat[order_idx][syngen_power_name_psat].time,
            ts_psat[order_idx][syngen_power_name_psat].values,
            label=syngen_power_name_psat
            + ", "
            + order_names_list[order_idx]
            + " (psat)",
            linestyle="--",
        )

    plt.xlim(view_time_interval)
    plt.xlabel("time (s)")
    plt.ylabel("torque (p.u)")
    plt.legend()

## Errors and Assertions

In [ ]:
for order_idx in range(num_orders):
    print("{} order:".format(order_names_list[order_idx]))

    rmse_gen_1 = ts_psat[order_idx]["p_Syn_3"].rmse(
        ts_psat[order_idx]["p_Syn_3"], ts_dpsim[order_idx]["GEN1.Te"]
    )
    print("{}: {}".format("GEN1", rmse_gen_1))
    assert rmse_gen_1 < 1e-2

    rmse_gen_2 = ts_psat[order_idx]["p_Syn_1"].rmse(
        ts_psat[order_idx]["p_Syn_1"], ts_dpsim[order_idx]["GEN2.Te"]
    )
    print("{}: {}".format("GEN2", rmse_gen_2))
    assert rmse_gen_2 < 1e-2

    rmse_gen_3 = ts_psat[order_idx]["p_Syn_2"].rmse(
        ts_psat[order_idx]["p_Syn_2"], ts_dpsim[order_idx]["GEN3.Te"]
    )
    print("{}: {}".format("GEN3", rmse_gen_3))
    assert rmse_gen_3 < 1e-2